In [1]:
import sys
sys.path.insert(0, '/home/lshtanko/Programming/another-fem-framework/')

In [2]:
from common import helpers
from common.tests import partial_diff_test

from common import polynom_factory
import numpy as np
from itertools import product

In [10]:
sizes = [(0.,float(2**i)) for i in range(1)]

for size in sizes:
    mass_matrix = polynom_factory.local_funcfunc_matrix(order=3, dim=2, distribution='globatto', size=size)
    gradgrad = polynom_factory.local_gradgrad_matrix(order=3, distribution='globatto', dim=2, size=size)
    gradfunc = polynom_factory.local_gradfunc_matrix(order=3, distribution='globatto', dim=2, size=size)[1][0]
    print(size)
    for (point,root) in gradgrad[1].items():
        print(point)
        partial_diff_test(matrix=gradgrad[0], point=point, pointnum=root, pointdict=gradgrad[1], mass_matrix=mass_matrix)

(0.0, 1.0)
(0.0, 1.0)
Mass coef: 0.006944442546594612
X derivative: 0.08333331574325438
Y derivative: -0.08333332639852925
XX derivative: -0.006944442548908598
YY derivative: -0.006944441272787363
XY_derivative: -9.236144073512809e-09

(0.2763932, 1.0)
Mass coef: 0.034722216994812774
X derivative: -1.7661245348132848e-09
Y derivative: -0.4166666831093994
XX derivative: -0.0347222157180124
YY derivative: -0.034722210602367354
XY_derivative: 1.0462534463795414e-08

(0.0, 0.72360682)
Mass coef: 0.03472221986082208
X derivative: 0.41666666416437936
Y derivative: 1.0911816589187495e-11
XX derivative: -0.03472221985335758
YY derivative: -0.03472221985286034
XY_derivative: 1.3864798652923893e-08

(0.72360682, 0.0)
Mass coef: 0.03472221985651801
X derivative: 5.608777331467252e-12
Y derivative: 0.4166666641650738
XX derivative: -0.034722219854031316
YY derivative: -0.034722219853545434
XY_derivative: 1.387056490845251e-08

(0.0, 0.2763932)
Mass coef: 0.0347222221683782
X derivative: 0.41666669

In [12]:
polynom_factory.glob_weights(order=2, size=(0,1))

array([ 0.33333334,  1.33333337,  0.33333334], dtype=float32)

In [11]:
polynom_factory.glob_weights(order=3, size=(0,1))

array([ 0.16666667,  0.83333331,  0.83333331,  0.16666667], dtype=float32)

In [5]:
polynom_factory.gauss_lobatto(n=3)

([-1, 0, 1], [0.3333333333, 1.333333333, 0.3333333333])

In [6]:
polynom_factory.glob_points(order=2, size=(0,4))

array([ 0.,  2.,  4.], dtype=float32)